In [1]:
import re, math, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler, QuantileTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from sklearn.isotonic import IsotonicRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import networkx as nx

from pyod.models.hbos import HBOS
from pyod.models.copod import COPOD

np.random.seed(42)

In [2]:
df = pd.read_excel('data/final_risk_data.xlsx')

# изменение типов данных
df['date'] = pd.to_datetime(df['date']).dt.normalize()
for i in ['debit_amount', 'credit_amount']:
    if i in df.columns:
        df[i] = pd.to_numeric(df[i], errors='coerce')
        
# создание колонки с суммой операции
df["amount"] = np.where(df["debit_amount"].notna(), df["debit_amount"], df["credit_amount"])
df["debit_amount"] = df["debit_amount"].fillna(0.0)
df["credit_amount"] = df["credit_amount"].fillna(0.0)

# созжание временных признаков
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
df['month'] = df['date'].dt.month
df["weekofyear"]   = df["date"].dt.isocalendar().week.astype(int)

# Суточные фичи
df['daily_total_debit'] = df.groupby(['debit_inn', 'date'])['debit_amount'].transform('sum')
df['daily_total_credit'] = df.groupby(['credit_inn', 'date'])['credit_amount'].transform('sum')
# посчет транзакций за сутки
df["daily_debit_transaction_count"]    = df.groupby(["debit_inn","date"])["debit_amount"].transform(lambda s: (s > 0).sum())
df["daily_credit_transaction_count"]    = df.groupby(["credit_inn","date"])["credit_amount"].transform(lambda s: (s > 0).sum())
# уникальные отправители и получатели в сутки
df["unique_recipients_per_day"] = df.groupby(["debit_inn","date"])["credit_inn"].transform("nunique")
df["unique_receivers_per_day"] = df.groupby(["credit_inn","date"])["debit_inn"].transform("nunique")

# Доля текущей операции в суточном объёме
df["daily_debit_percent"] = (df["debit_amount"] / df["daily_total_debit"].replace(0, np.nan)).fillna(0.0)
df["daily_credit_percent"] = (df["credit_amount"] / df["daily_total_credit"].replace(0, np.nan)).fillna(0.0)

# Интервалы между операциями (в днях)
df["days_since_last_txn_debit"]  = df.groupby("debit_inn")["date"].diff().dt.days.fillna(9999)
df["days_since_last_txn_credit"] = df.groupby("credit_inn")["date"].diff().dt.days.fillna(9999)

def add_rolling_side(df: pd.DataFrame,
                     side: str,
                     amt_col: str,
                     windows=(7, 14, 30, 90)) -> pd.DataFrame:
    """
    Создает роллинги по суммам/кол-ву транзакций за окна windows для заданной стороны.
    side: 'debit' или 'credit'
    amt_col: имя столбца с суммой для этой стороны (например, 'debit_amount' / 'credit_amount')
    """
    inn_col = f"{side}_inn"
    out_parts = []

    # Берем только нужные колонки (ускоряет groupby)
    need_cols = [inn_col, "date", amt_col]
    sub_all = df[need_cols].copy()
    sub_all[amt_col] = sub_all[amt_col].fillna(0.0)

    for inn, sub in sub_all.groupby(inn_col, sort=False):
        # суточная агрегация по аккаунту
        daily = (sub.groupby("date", as_index=True)[amt_col]
                   .sum()
                   .to_frame("amt_day")
                   .sort_index())

        # непрерывный календарный индекс (чтобы окна не "сжимались")
        idx = pd.date_range(daily.index.min(), daily.index.max(), freq="D")
        daily = daily.reindex(idx, fill_value=0.0)
        daily.index.name = "date"

        # индикатор "была ли операция в день" (для роллинга количества)
        daily["tx_day"] = (daily["amt_day"] > 0).astype(int)

        # роллинги
        for W in windows:
            daily[f"{side}_roll_sum_{W}d"]  = daily["amt_day"].rolling(W, min_periods=1).sum()
            daily[f"{side}_roll_cnt_{W}d"]  = daily["tx_day"].rolling(W, min_periods=1).sum()
            daily[f"{side}_roll_mean_{W}d"] = daily["amt_day"].rolling(W, min_periods=1).mean()
            daily[f"{side}_roll_std_{W}d"]  = daily["amt_day"].rolling(W, min_periods=1).std().fillna(0.0)
            daily[f"{side}_roll_p95_{W}d"]  = daily["amt_day"].rolling(W, min_periods=1).quantile(0.95)

        daily = daily.reset_index()
        daily[inn_col] = inn
        out_parts.append(daily[["date", inn_col] + [c for c in daily.columns if c.startswith(f"{side}_roll_")]])

    rolls = pd.concat(out_parts, ignore_index=True)
    return df.merge(rolls, on=["date", inn_col], how="left")



# Новые роллинги 7/14/30/90 для дебета и кредита
WINDOWS = (7, 14, 30, 90)
df = add_rolling_side(df, side="debit",  amt_col="debit_amount",  windows=WINDOWS)
df = add_rolling_side(df, side="credit", amt_col="credit_amount", windows=WINDOWS)

# ==== 1. Всплески сумм (amount_spike_ratio_7d) ====
df["debit_amount_spike_ratio_7d"]  = df["debit_roll_sum_7d"]  / (df["debit_roll_sum_30d"]/4 + 1e-6)
df["credit_amount_spike_ratio_7d"] = df["credit_roll_sum_7d"] / (df["credit_roll_sum_30d"]/4 + 1e-6)

# ==== 2. Всплески активности (tx_rate_spike_7d) ====
df["debit_tx_rate_spike_7d"]  = df["debit_roll_cnt_7d"]  / (df["debit_roll_cnt_30d"]/4 + 1e-6)
df["credit_tx_rate_spike_7d"] = df["credit_roll_cnt_7d"] / (df["credit_roll_cnt_30d"]/4 + 1e-6)

# ==== 3. Fan-out / Fan-in (уникальные контрагенты) ====
# Для дебета — fan-out (сколько разных получателей на операцию)
df["debit_fan_out_ratio"] = df["unique_recipients_per_day"] / (df["daily_debit_transaction_count"] + 1e-6)

# Для кредита — fan-in (сколько разных отправителей на операцию)
df["credit_fan_in_ratio"] = (
    df.groupby(["credit_inn", "date"])["debit_inn"].transform("nunique") /
    (df["daily_credit_transaction_count"] + 1e-6)
)

# ==== 4. Дисбаланс потоков (in_out_ratio_30d) ====
df["in_out_ratio_30d"] = (df["credit_roll_sum_30d"] + 1e-6) / (df["debit_roll_sum_30d"] + 1e-6)

# ==== 5. Волатильность объёмов (amount_volatility_30d) ====
df["debit_amount_volatility_30d"]  = df["debit_roll_std_30d"]  / (df["debit_roll_mean_30d"]  + 1e-6)
df["credit_amount_volatility_30d"] = df["credit_roll_std_30d"] / (df["credit_roll_mean_30d"] + 1e-6)


# Круглые суммы
df["round_10k"]  = ((df["amount"] % 10000)  == 0).astype(int)
df["round_100k"] = ((df["amount"] % 100000) == 0).astype(int)
df["round_large_amount"] = ((df["round_10k"]==1) | (df["round_100k"]==1)).astype(int)

df.head()

,date,debit_account,debit_name,debit_inn,credit_account,credit_name,credit_inn,debit_amount,credit_amount,purpose,...,debit_tx_rate_spike_7d,credit_tx_rate_spike_7d,debit_fan_out_ratio,credit_fan_in_ratio,in_out_ratio_30d,debit_amount_volatility_30d,credit_amount_volatility_30d,round_10k,round_100k,round_large_amount
0,2019-01-09,ded93f97f389bf2c,b736fcdbf591b1c2,d877722ca4e40f98,0c00203649ed677a,e19cc80da09f445f,8d9e0be733f77f1c,10.0,0.0,комиссия внутри сбербанка за пп/пт через дбо с...,...,3.999984,0.000000,1.0,1000000.0,3.333322e-13,0.000000,0.0,0,0,0
1,2019-01-09,ded93f97f389bf2c,b736fcdbf591b1c2,d877722ca4e40f98,7682fd5c32e028f5,796a88c244ffcc0a,6931e23e98703aa9,3000000.0,0.0,оплата по счету № 28 от date_9f241b636025 по д...,...,3.999984,0.000000,1.0,1000000.0,3.333322e-13,0.000000,0.0,1,1,1
2,2019-01-10,ded93f97f389bf2c,b736fcdbf591b1c2,d877722ca4e40f98,0c00203649ed677a,e19cc80da09f445f,8d9e0be733f77f1c,10.0,0.0,комиссия внутри сбербанка за пп/пт через дбо с...,...,3.999992,0.000000,1.0,1000000.0,3.328560e-13,1.410173,0.0,0,0,0
3,2019-01-10,ded93f97f389bf2c,b736fcdbf591b1c2,d877722ca4e40f98,e63cf19b76230a3d,21a73421ca00ae87,eed8b1f54ed4366c,4282.0,0.0,оплата по договору электроэнергия по дог.№1124...,...,3.999992,0.000000,1.0,1000000.0,3.328560e-13,1.410173,0.0,0,0,0
4,2019-01-10,d658b4e51c5a5df5,f7185cc66db08da5,ad57c94e5b8df8f6,ded93f97f389bf2c,3e368a63959d285e,d877722ca4e40f98,0.0,1000000.0,перевод денежных средств по договору займа № ч...,...,0.000000,3.999984,1000000.0,1.0,3.000000e+12,0.000000,0.0,1,1,1


In [3]:
# ============================================================
#  Анализ "Назначения платежа": TF-IDF (char 3–5) + стоп-слова риска
# ============================================================

import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# ------------------------------------------------------------
# 1) Регулярные паттерны высокого риска
# ------------------------------------------------------------
STOP_HIGH_PATTERNS = [
    r"\bзайм\w*\b", r"\bвозврат\W*(займ|долг)\w*\b", r"\bпогашен\w*\W*(займ|долг)\w*\b",
    r"\bдолг\w*\b", r"\bоплат\w*\W*(займ|долг)\w*\b",
    r"\bкрипт\w*\b", r"\bбиткоин\w*\b", r"\busdt\b", r"\bbtc\b", r"\bcoin\b",
    r"\bбирж\w*\b", r"\bобмен\w*\b", r"\bвалют\w*\b",
    r"\bналич\w*\b", r"\bобнал\w*\b", r"\bвыдач\w*\W*налич\w*\b",
    r"\bперевод\w*\W*(на|с)\W*карт\w*\b", r"\bперевод\w*\W*физ\w*\b",
    r"\bперевод\w*\W*родствен\w*\б", r"\bперевод\w*\W*средств\b",
    r"\bбез\W*договор\w*\b",
    r"\bпожертв\w*\b", r"\bблаготвор\w*\б", r"\bдарен\w*\b",
    r"\bагентск\w*\W*вознагражд\w*\б", r"\bкомисси\w*\W*вознагражд\w*\b",
    r"\bвознагражд\w*\b",
    r"\bцесс\w*\б", r"\bпоручител\w*\b", r"\bзалог\w*\b",
    r"\bофшор\w*\б", r"\bиностран\w*\W*перевод\w*\b", r"\bswift\b",
    r"\bличн\w*\W*нужд\w*\б", r"\bпередач\w*\W*актив\w*\б",
    r"\bвклад\w*\б", r"\bдепозит\w*\б",
]
STOP_HIGH_RE = re.compile("|".join(STOP_HIGH_PATTERNS), flags=re.IGNORECASE)

# ------------------------------------------------------------
# 2) Очистка текста
# ------------------------------------------------------------
def clean_text(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"[^a-zа-я0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["purpose_clean"] = df["purpose"].apply(clean_text)

# ------------------------------------------------------------
# 3) Метка наличия стоп-слов риска
# ------------------------------------------------------------
df["purpose_stopword_high"] = df["purpose_clean"].str.contains(STOP_HIGH_RE, na=False)

# ------------------------------------------------------------
# 4) TF-IDF (char-level 3–5-граммы) + SVD
# ------------------------------------------------------------
texts = df["purpose_clean"].astype(str).tolist()

tfidf = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3, 5),
    min_df=1
)
X_tfidf = tfidf.fit_transform(texts)
print(f"TF-IDF матрица: {X_tfidf.shape[0]} × {X_tfidf.shape[1]}")

# SVD для сжатия (например, до 50 компонент)
svd_k_target = 50
max_svd = max(1, min(X_tfidf.shape[0] - 1, X_tfidf.shape[1] - 1))
n_svd = min(svd_k_target, max_svd)

svd = TruncatedSVD(n_components=n_svd, random_state=42)
X_svd = svd.fit_transform(X_tfidf)

for i in range(X_svd.shape[1]):
    df[f"purpose_svd_{i+1}"] = X_svd[:, i]

expl_var = svd.explained_variance_ratio_.sum()
print(f"SVD explained variance (k={X_svd.shape[1]}): {expl_var:.3f}")

# ------------------------------------------------------------
# 5) Результат
# ------------------------------------------------------------
svd_cols = [c for c in df.columns if c.startswith("purpose_svd_")]
cols_show = ["purpose", "purpose_stopword_high"] + svd_cols

print(f"\n✅ Добавлены признаки: {len(svd_cols)} SVD-компонент + метка стоп-слов")
print(df[cols_show].head())


TF-IDF матрица: 2688 × 45426
SVD explained variance (k=50): 0.545

✅ Добавлены признаки: 50 SVD-компонент + метка стоп-слов
                                             purpose  purpose_stopword_high  \
0  комиссия внутри сбербанка за пп/пт через дбо с...                  False   
1  оплата по счету № 28 от date_9f241b636025 по д...                  False   
2  комиссия внутри сбербанка за пп/пт через дбо с...                  False   
3  оплата по договору электроэнергия по дог.№1124...                  False   
4  перевод денежных средств по договору займа № ч...                   True   

   purpose_svd_1  purpose_svd_2  purpose_svd_3  purpose_svd_4  purpose_svd_5  \
0       0.708416      -0.202923      -0.110215      -0.029243       0.142140   
1       0.122677       0.236405      -0.200400      -0.017372      -0.097021   
2       0.602413      -0.212909      -0.103785      -0.020422       0.124845   
3       0.108051       0.160672      -0.135616      -0.000770      -0.035201   
4

In [4]:
df.columns

Index(['date', 'debit_account', 'debit_name', 'debit_inn', 'credit_account',
       'credit_name', 'credit_inn', 'debit_amount', 'credit_amount', 'purpose',
       ...
       'purpose_svd_41', 'purpose_svd_42', 'purpose_svd_43', 'purpose_svd_44',
       'purpose_svd_45', 'purpose_svd_46', 'purpose_svd_47', 'purpose_svd_48',
       'purpose_svd_49', 'purpose_svd_50'],
      dtype='object', length=130)

In [32]:
# ============================================================
#  Обнаружение аномалий для ИНН (debit+credit)
#  Несколько моделей sklearn: IsolationForest, LOF, OneClassSVM, EllipticEnvelope
#  + фильтр: анализ только ИНН с >=20 транзакциями
# ============================================================

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
import warnings
warnings.filterwarnings("ignore")

print(f"Всего транзакций: {len(df):,}")

# ---------- 1) Агрегация по ИНН ----------
def agg_side(df, inn_col, prefix):
    aggs = df.groupby(inn_col).agg(
        tx_count=('date', 'size'),
        amount_sum=('amount', 'sum'),
        amount_mean=('amount', 'mean'),
        amount_std=('amount', 'std'),
        roll_cnt_30d_mean=(f'{prefix}_roll_cnt_30d','mean'),
        roll_mean_30d_mean=(f'{prefix}_roll_mean_30d','mean'),
        roll_std_30d_mean=(f'{prefix}_roll_std_30d','mean'),
        spike_ratio_7d_mean=(f'{prefix}_amount_spike_ratio_7d','mean'),
        tx_rate_spike_7d_mean=(f'{prefix}_tx_rate_spike_7d','mean'),
        vol_30d_mean=(f'{prefix}_amount_volatility_30d','mean'),
    )

    # добавим префикс ко всем колонкам, чтобы избежать пересечения
    aggs = aggs.add_prefix(f"{prefix}_")
    return aggs

agg_deb = agg_side(df, 'debit_inn', 'debit')
agg_cred = agg_side(df, 'credit_inn', 'credit')
agg_all = agg_deb.join(agg_cred, how='outer').fillna(0.0)

# самопереводы конкретно и только для пересечения
self_tx = (df.loc[df['debit_inn'] == df['credit_inn'], 'debit_inn']
             .value_counts()
             .rename('self_tx_count'))
agg_all = agg_all.join(self_tx, how='left').fillna({'self_tx_count': 0})

# корректный total_tx_count без двойного учёта
agg_all['total_tx_count'] = (
    agg_all.get('debit_tx_count', 0).astype(float)
  + agg_all.get('credit_tx_count', 0).astype(float)
  - agg_all['self_tx_count'].astype(float)
)

# ---------- 1.1) Сетевые показатели ----------
extra = df.groupby('debit_inn').agg(out_unique=('credit_inn','nunique'))
extra2 = df.groupby('credit_inn').agg(in_unique=('debit_inn','nunique'))
agg_all = agg_all.join(extra, how='left').join(extra2, how='left').fillna(0)

agg_all['fan_out_ratio'] = agg_all['out_unique'] / (agg_all['debit_tx_count']+1e-6)
agg_all['fan_in_ratio']  = agg_all['in_unique']  / (agg_all['credit_tx_count']+1e-6)
agg_all['in_out_ratio']  = (agg_all['credit_amount_sum']+1e-6)/(agg_all['debit_amount_sum']+1e-6)

# ---------- 1.2) Фильтр: только ИНН с >=20 транзакциями ----------

agg_all_filtered = agg_all[agg_all["total_tx_count"] >= 20].copy()
print(f"Отобрано {len(agg_all_filtered)} ИНН с >=20 транзакциями из {len(agg_all)}")

if len(agg_all_filtered) == 0:
    raise RuntimeError("❌ Нет ИНН с достаточным количеством транзакций для анализа.")

# ---------- 1.3) Масштабирование ----------
X = agg_all_filtered.fillna(0.0).replace([np.inf,-np.inf],0.0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Агрегировано {len(agg_all_filtered):,} ИНН, {X_scaled.shape[1]} фичей")

# ---------- 2) Модели ----------
models = {
    "IsolationForest": IsolationForest(n_estimators=400, contamination=0.03, random_state=42, n_jobs=-1),
    "LocalOutlierFactor": LocalOutlierFactor(n_neighbors=20, contamination=0.03, novelty=True),
    "OneClassSVM": OneClassSVM(kernel="rbf", gamma=0.05, nu=0.03),
    "EllipticEnvelope": EllipticEnvelope(contamination=0.03, random_state=42)
}

results = {}
scores_all = pd.DataFrame(index=agg_all_filtered.index)

for name, model in models.items():
    print(f"⏳ Обучаем {name} ...")
    model.fit(X_scaled)
    if hasattr(model, "score_samples"):
        raw = -model.score_samples(X_scaled)
    else:
        raw = -model.decision_function(X_scaled)
    s = (raw - raw.min()) / (raw.max() - raw.min() + 1e-6)
    scores_all[name+"_score"] = s
    results[name] = {
        "mean": float(s.mean()),
        "std": float(s.std()),
        "top5_mean": float(np.mean(np.sort(s)[-5:])),
        "threshold_97": float(np.quantile(s,0.95))
    }
    print(f"  min={s.min():.3f} max={s.max():.3f} mean={s.mean():.3f}")

# ---------- 3) Композитный скор ----------
scores_all["graph_anomaly_score"] = scores_all.mean(axis=1)
thr = np.quantile(scores_all["graph_anomaly_score"],0.97)
scores_all["anomaly_flag"] = (scores_all["graph_anomaly_score"]>=thr).astype(int)

agg_all_filtered = pd.concat([agg_all_filtered, scores_all], axis=1)

print("\n=== Итоговые пороги и статистика ===")
for k,v in results.items():
    print(f"{k:20s}  mean={v['mean']:.3f}  thr97={v['threshold_97']:.3f}")

print(f"\nСводный порог: {thr:.3f} | Аномальных ИНН: {agg_all_filtered['anomaly_flag'].sum()} из {len(agg_all_filtered)}")

# ---------- 4) Топ аномальных ИНН ----------
top = agg_all_filtered.sort_values("graph_anomaly_score", ascending=False).head(20)
display(top[["graph_anomaly_score","anomaly_flag"] + [c for c in top.columns if "ratio" in c or "sum" in c or "count" in c]].head(20))

# ---------- 5) Сохранение ----------
agg_all_filtered.to_csv("inn_anomaly_scores_filtered.csv")
print("\n✅ Результаты сохранены: inn_anomaly_scores_filtered.csv (ИНН с >=20 транзакциями, с флагами и score)")


Всего транзакций: 2,688
Отобрано 20 ИНН с >=20 транзакциями из 111
Агрегировано 20 ИНН, 27 фичей
⏳ Обучаем IsolationForest ...
  min=0.000 max=1.000 mean=0.198
⏳ Обучаем LocalOutlierFactor ...
  min=0.000 max=0.000 mean=0.000
⏳ Обучаем OneClassSVM ...
  min=0.000 max=1.000 mean=0.791
⏳ Обучаем EllipticEnvelope ...
  min=0.000 max=1.000 mean=0.707

=== Итоговые пороги и статистика ===
IsolationForest       mean=0.198  thr97=0.594
LocalOutlierFactor    mean=0.000  thr97=0.000
OneClassSVM           mean=0.791  thr97=0.997
EllipticEnvelope      mean=0.707  thr97=1.000

Сводный порог: 0.686 | Аномальных ИНН: 1 из 20


,graph_anomaly_score,anomaly_flag,debit_tx_count,debit_amount_sum,debit_spike_ratio_7d_mean,credit_tx_count,credit_amount_sum,credit_spike_ratio_7d_mean,self_tx_count,total_tx_count,fan_out_ratio,fan_in_ratio,in_out_ratio
d877722ca4e40f98,0.744333,1,2152.0,6.771678e+08,1.271407,560.0,6.803307e+08,1.292062,24.0,2688.0,0.045074,0.067857,1.004671e+00
8d9e0be733f77f1c,0.642451,0,223.0,3.538290e+08,0.000000,983.0,1.132932e+08,0.000000,0.0,1206.0,0.004484,0.001017,3.201919e-01
6931e23e98703aa9,0.623952,0,0.0,0.000000e+00,0.000000,256.0,3.934273e+08,0.000000,0.0,256.0,0.000000,0.003906,3.934273e+14
9d8f54ae3b88911b,0.585435,0,32.0,4.984884e+07,0.000000,7.0,2.280279e+07,0.000000,0.0,39.0,0.031250,0.142857,4.574388e-01
80799e1a6646b6dc,0.573548,0,9.0,7.698837e+07,0.000000,172.0,3.239357e+06,0.000000,0.0,181.0,0.111111,0.005814,4.207593e-02
6ef5667d7851dcbc,0.559121,0,20.0,2.320000e+07,0.000000,1.0,3.000000e+06,0.000000,0.0,21.0,0.050000,0.999999,1.293103e-01
1b21a7645f64d170,0.547137,0,1.0,2.357825e+06,0.000000,39.0,5.317691e+06,0.000000,0.0,40.0,0.999999,0.025641,2.255337e+00
659207c64fee1c66,0.543846,0,60.0,1.820000e+07,0.000000,1.0,1.000000e+05,0.000000,0.0,61.0,0.016667,0.999999,5.494505e-03
5ae9eeb0d29bc5c0,0.478128,0,43.0,1.110000e+07,0.000000,0.0,0.000000e+00,0.000000,0.0,43.0,0.023256,0.000000,9.009009e-14
eed8b1f54ed4366c,0.473275,0,0.0,0.000000e+00,0.000000,82.0,1.228693e+06,0.000000,0.0,82.0,0.000000,0.012195,1.228693e+12



✅ Результаты сохранены: inn_anomaly_scores_filtered.csv (ИНН с >=20 транзакциями, с флагами и score)


In [45]:
# ============================================================
#  Обнаружение аномалий для ИНН (debit+credit)
#  Модели: IsolationForest, LOF, OneClassSVM, EllipticEnvelope
#  + фильтр: анализ только ИНН с >=20 транзакциями
#  + топ-100 подозрительных ТРАНЗАКЦИЙ по лучшей модели
# ============================================================

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
import warnings
warnings.filterwarnings("ignore")

print(f"Всего транзакций: {len(df):,}")

# ---------- 1) Агрегация по ИНН ----------
def agg_side(df, inn_col, prefix):
    aggs = df.groupby(inn_col).agg(
        tx_count=('date', 'size'),
        amount_sum=('amount', 'sum'),
        amount_mean=('amount', 'mean'),
        amount_std=('amount', 'std'),
        roll_cnt_30d_mean=(f'{prefix}_roll_cnt_30d','mean'),
        roll_mean_30d_mean=(f'{prefix}_roll_mean_30d','mean'),
        roll_std_30d_mean=(f'{prefix}_roll_std_30d','mean'),
        spike_ratio_7d_mean=(f'{prefix}_amount_spike_ratio_7d','mean'),
        tx_rate_spike_7d_mean=(f'{prefix}_tx_rate_spike_7d','mean'),
        vol_30d_mean=(f'{prefix}_amount_volatility_30d','mean'),
    )
    aggs = aggs.add_prefix(f"{prefix}_")
    return aggs

agg_deb = agg_side(df, 'debit_inn', 'debit')
agg_cred = agg_side(df, 'credit_inn', 'credit')
agg_all = agg_deb.join(agg_cred, how='outer').fillna(0.0)

# самопереводы
self_tx = (df.loc[df['debit_inn'] == df['credit_inn'], 'debit_inn']
             .value_counts()
             .rename('self_tx_count'))
agg_all = agg_all.join(self_tx, how='left').fillna({'self_tx_count': 0})

# корректный total_tx_count без двойного учёта
agg_all['total_tx_count'] = (
    agg_all.get('debit_tx_count', 0).astype(float)
  + agg_all.get('credit_tx_count', 0).astype(float)
  - agg_all['self_tx_count'].astype(float)
)

# ---------- 1.1) Сетевые показатели ----------
extra  = df.groupby('debit_inn').agg(out_unique=('credit_inn','nunique'))
extra2 = df.groupby('credit_inn').agg(in_unique=('debit_inn','nunique'))
agg_all = agg_all.join(extra, how='left').join(extra2, how='left').fillna(0)

agg_all['fan_out_ratio'] = agg_all['out_unique'] / (agg_all['debit_tx_count']+1e-6)
agg_all['fan_in_ratio']  = agg_all['in_unique']  / (agg_all['credit_tx_count']+1e-6)
agg_all['in_out_ratio']  = (agg_all['credit_amount_sum']+1e-6)/(agg_all['debit_amount_sum']+1e-6)

# ---------- 1.2) Фильтр: только ИНН с >=20 транзакциями ----------
agg_all_filtered = agg_all[agg_all["total_tx_count"] >= 20].copy()
print(f"Отобрано {len(agg_all_filtered)} ИНН с >=20 транзакциями из {len(agg_all)}")
if len(agg_all_filtered) == 0:
    raise RuntimeError("❌ Нет ИНН с достаточным количеством транзакций для анализа.")

# ---------- 1.3) Масштабирование ----------
X = agg_all_filtered.replace([np.inf,-np.inf],0.0).fillna(0.0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"Агрегировано {len(agg_all_filtered):,} ИНН, {X_scaled.shape[1]} фичей")

# ---------- 2) Модели ----------
models = {
    "IsolationForest": IsolationForest(n_estimators=400, contamination=0.03, random_state=42, n_jobs=-1),
    "LocalOutlierFactor": LocalOutlierFactor(n_neighbors=20, contamination=0.03, novelty=True),
    "OneClassSVM": OneClassSVM(kernel="rbf", gamma=0.05, nu=0.03),
    "EllipticEnvelope": EllipticEnvelope(contamination=0.03, random_state=42)
}

results = {}
scores_all = pd.DataFrame(index=agg_all_filtered.index)

for name, model in models.items():
    print(f"⏳ Обучаем {name} ...")
    model.fit(X_scaled)
    # приводим к шкале: чем больше, тем аномальнее
    if hasattr(model, "score_samples"):
        raw = -model.score_samples(X_scaled)
    else:
        raw = -model.decision_function(X_scaled)
    raw = np.asarray(raw, dtype=float)
    s = (raw - raw.min()) / (raw.max() - raw.min() + 1e-6)
    scores_all[name+"_score"] = s
    results[name] = {
        "mean": float(s.mean()),
        "std": float(s.std()),
        "top5_mean": float(np.mean(np.sort(s)[-5:])),
        "threshold_97": float(np.quantile(s,0.97))
    }
    print(f"  min={s.min():.3f} max={s.max():.3f} mean={s.mean():.3f}")

# ---------- 3) Композитный скор и выбор лучшей модели ----------
scores_all["ensemble_score"] = scores_all.mean(axis=1)
thr = np.quantile(scores_all["ensemble_score"],0.97)
scores_all["anomaly_flag"] = (scores_all["ensemble_score"]>=thr).astype(int)

agg_all_filtered = pd.concat([agg_all_filtered, scores_all], axis=1)

print("\n=== Итоговые пороги и статистика ===")
for k,v in results.items():
    print(f"{k:20s}  mean={v['mean']:.3f}  thr97={v['threshold_97']:.3f}")
print(f"\nСводный порог (ансамбль, p97): {thr:.3f} | Аномальных ИНН: {agg_all_filtered['anomaly_flag'].sum()} из {len(agg_all_filtered)}")

# лучшая модель = максимальная корреляция с ансамблем (устойчивость)
corr_with_ens = scores_all.drop(columns=["ensemble_score","anomaly_flag"]).corrwith(scores_all["ensemble_score"])
best_model = corr_with_ens.idxmax() + "_score" if not corr_with_ens.empty else "IsolationForest_score"
if best_model not in scores_all.columns:
    best_model = "IsolationForest_score"
print(f"\n🎯 Лучшая модель для маппинга на транзакции: {best_model}")

# ---------- 4) Формирование транзакционного скора (гибрид) ----------
# маппинг скор-а лучшей модели по участникам
df = df.copy()
df["debit_score"]  = df["debit_inn"].map(agg_all_filtered[best_model])
df["credit_score"] = df["credit_inn"].map(agg_all_filtered[best_model])

# если нет текстовой метрики — создадим заглушку
if "purpose_risk_score" not in df.columns:
    df["purpose_risk_score"] = 0.0

# нормированная сумма (лог), чтобы разгладить масштаб
amount_norm = np.log1p(df["amount"].clip(lower=0)) / (np.log1p(df["amount"].clip(lower=0)).max() + 1e-9)

# локальный ранг суммы внутри отправителя (контраст внутри ИНН)
df["local_amount_rank"] = df.groupby("debit_inn")["amount"].rank(pct=True)

# гибридный транзакционный скор:
#   70% — риск участника (макс дебет/кредит),
#   20% — риск текста,
#   10% — объём/локальный контраст.
df["txn_risk_score"] = (
    0.7 * df[["debit_score", "credit_score"]].max(axis=1).fillna(0) +
    0.2 * df["purpose_risk_score"].fillna(0) +
    0.1 * (0.5*amount_norm.fillna(0) + 0.5*df["local_amount_rank"].fillna(0))
)

df["txn_risk_score"] = (df["txn_risk_score"] - df["txn_risk_score"].min()) / (
    df["txn_risk_score"].max() - df["txn_risk_score"].min() + 1e-9
)

# флаг top-3% по транзакциям
thr_txn = np.quantile(df["txn_risk_score"].dropna(), 0.97)
df["txn_anomaly_flag"] = (df["txn_risk_score"] >= thr_txn).astype(int)
print(f"\nПорог транзакций (p97): {thr_txn:.3f} | Аномальных транзакций: {df['txn_anomaly_flag'].sum()} из {len(df)}")

# ---------- 5) Топ-100 подозрительных транзакций ----------
cols_show = [
    "date","debit_inn","credit_inn","amount","purpose",
    "purpose_risk_score","debit_score","credit_score",
    "txn_risk_score","txn_anomaly_flag"
]
cols_show = [c for c in cols_show if c in df.columns]

top_n = 2000
top_txn = df.sort_values("txn_risk_score", ascending=False).head(top_n).copy()
print(f"\n=== Топ-{top_n} подозрительных транзакций ({best_model}) ===")
print(top_txn[cols_show].head(20))  # в вывод — первые 20 строк для краткости

# сохранение файлов
agg_all_filtered.to_csv("inn_anomaly_scores_filtered.csv", index=False, encoding="utf-8-sig")
top_txn[cols_show].to_csv(f"top_{top_n}_anomalous_transactions_{best_model}.csv", index=False, encoding="utf-8-sig")

print("\n✅ Сохранено:")
print("  • inn_anomaly_scores_filtered.csv  — ИНН с >=20 транзакциями (скоры, флаги)")
print(f"  • top_{top_n}_anomalous_transactions_{best_model}.csv  — топ-{top_n} подозрительных транзакций")


Всего транзакций: 2,688
Отобрано 20 ИНН с >=20 транзакциями из 111
Агрегировано 20 ИНН, 27 фичей
⏳ Обучаем IsolationForest ...
  min=0.000 max=1.000 mean=0.198
⏳ Обучаем LocalOutlierFactor ...
  min=0.000 max=0.000 mean=0.000
⏳ Обучаем OneClassSVM ...
  min=0.000 max=1.000 mean=0.791
⏳ Обучаем EllipticEnvelope ...
  min=0.000 max=1.000 mean=0.707

=== Итоговые пороги и статистика ===
IsolationForest       mean=0.198  thr97=0.756
LocalOutlierFactor    mean=0.000  thr97=0.000
OneClassSVM           mean=0.791  thr97=0.998
EllipticEnvelope      mean=0.707  thr97=1.000

Сводный порог (ансамбль, p97): 0.686 | Аномальных ИНН: 1 из 20

🎯 Лучшая модель для маппинга на транзакции: IsolationForest_score

Порог транзакций (p97): 0.881 | Аномальных транзакций: 81 из 2688

=== Топ-2000 подозрительных транзакций (IsolationForest_score) ===
           date         debit_inn        credit_inn        amount  \
1899 2021-07-06  8d9e0be733f77f1c  d877722ca4e40f98  1.000000e+08   
1898 2021-07-06  d877722c

In [47]:
df['debit_score'].value_counts()

debit_score
0.999998    2152
0.572508     223
0.183957      60
0.136701      43
0.351055      32
0.075095      24
0.143359      23
0.236536      20
0.316077       9
0.045881       8
0.053857       7
0.197020       1
Name: count, dtype: int64